In [1]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd

## Calculate area from 3D vertice

In [2]:
#determinant of matrix a
def det(a):
    return a[0][0]*a[1][1]*a[2][2] + a[0][1]*a[1][2]*a[2][0] + a[0][2]*a[1][0]*a[2][1] - a[0][2]*a[1][1]*a[2][0] - a[0][1]*a[1][0]*a[2][2] - a[0][0]*a[1][2]*a[2][1]

#unit normal vector of plane defined by points a, b, and c
def unit_normal(a, b, c):
    x = det([[1,a[1],a[2]],
             [1,b[1],b[2]],
             [1,c[1],c[2]]])
    y = det([[a[0],1,a[2]],
             [b[0],1,b[2]],
             [c[0],1,c[2]]])
    z = det([[a[0],a[1],1],
             [b[0],b[1],1],
             [c[0],c[1],1]])
    magnitude = (x**2 + y**2 + z**2)**.5
    return (x/magnitude, y/magnitude, z/magnitude)

#dot product of vectors a and b
def dot(a, b):
    return a[0]*b[0] + a[1]*b[1] + a[2]*b[2]

#cross product of vectors a and b
def cross(a, b):
    x = a[1] * b[2] - a[2] * b[1]
    y = a[2] * b[0] - a[0] * b[2]
    z = a[0] * b[1] - a[1] * b[0]
    return (x, y, z)

#area of polygon poly
def area(poly):
    if len(poly) < 3: # not a plane - no area
        return 0

    total = [0, 0, 0]
    for i in range(len(poly)):
        vi1 = poly[i]
        if i is len(poly)-1:
            vi2 = poly[0]
        else:
            vi2 = poly[i+1]
        prod = cross(vi1, vi2)
        total[0] += prod[0]
        total[1] += prod[1]
        total[2] += prod[2]
    result = dot(total, unit_normal(poly[0], poly[1], poly[2]))
    return abs(result/2)

## Read red nodes

In [3]:
name_to_id = {}
work_folder = 'SF_100'
scene_folder = 'scene-json'
res_json = {}
with open(os.path.join(scene_folder, 'meta', 'sf_dt_98_red_nodes_data_exchanges.json'), 'r') as f:
    eplus_meta = json.load(f)
    for bldg in eplus_meta['features']:
        bldg_id = bldg['id']
        res_json[bldg_id] = {}
        name_to_id[bldg_id] = {}
        total_area = 0
        for surf in bldg['properties']['objects']:
            surf_id = surf['id']
            res_json[bldg_id][surf_id] = {}
            name_to_id[bldg_id][surf['properties']['name']] = surf_id
            res_json[bldg_id][surf_id]['name'] = surf['properties']['name']
            res_json[bldg_id][surf_id]['v'] = surf['properties']['v']
            res_json[bldg_id][surf_id]['area'] = area(surf['properties']['v'])
            total_area += res_json[bldg_id][surf_id]['area']
        for surf in bldg['properties']['objects']:
            surf_id = surf['id']
            res_json[bldg_id][surf_id]['area_to_zone'] = res_json[bldg_id][surf_id]['area'] / total_area

In [4]:
res_json['bldg_3981169_2289']['surf_1']

{'name': 'S1-office-core-wall-0',
 'v': [[377.9300000001956, 529.8599999999627, 5.22],
  [377.9300000001956, 529.8599999999627, 0.0],
  [375.0800000001024, 549.5999999999534, 0.0],
  [375.0800000001024, 549.5999999999534, 5.22]],
 'area': 104.11120862253202,
 'area_to_zone': 0.022220621277582762}

## Read EnergyPlus output CSV files

In [5]:
# example file
result_name = os.path.join(work_folder, 'bldg_3981169_2289', 'eplusout-r2.csv') ## TODO
df_eplus_result = pd.read_csv(result_name, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

In [6]:
df_eplus_result.keys()

Index(['Date/Time',
       'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
       'Environment:Site Outdoor Air Dewpoint Temperature [C](Hourly)',
       'Environment:Site Outdoor Air Wetbulb Temperature [C](Hourly)',
       'Environment:Site Outdoor Air Relative Humidity [%](Hourly)',
       'Environment:Site Outdoor Air Barometric Pressure [Pa](Hourly)',
       'Environment:Site Wind Speed [m/s](Hourly)',
       'Environment:Site Wind Direction [deg](Hourly)',
       'S1 OFFICE CORE WALL 0:Surface Outside Face Temperature [C](Hourly)',
       'S1 OFFICE CORE WALL 0:Surface Outside Face Convection Heat Transfer Coefficient [W/m2-K](Hourly)',
       ...
       'SUB SURFACE 45:Surface Outside Face Convection Heat Transfer Coefficient [W/m2-K](Hourly)',
       'SUB SURFACE 45:Surface Outside Face Heat Emission to Air Rate [W](Hourly)',
       'SUB SURFACE 46:Surface Outside Face Temperature [C](Hourly)',
       'SUB SURFACE 46:Surface Outside Face Convection Heat Transfer

In [7]:
time_stamps = set()
for bldg in res_json:
    result_name_1 = os.path.join(work_folder, bldg, 'eplusout-r2.csv')  ## TODo
    df_eplus_result = pd.read_csv(result_name_1, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
    for col in df_eplus_result:
        df_eplus_result[col.strip()] = df_eplus_result[col]
    
    if not 'SimHVAC:HVAC System Total Heat Rejection Energy [J](Hourly)' in df_eplus_result:
        print(bldg + " not enough")
        continue
    for surf in res_json[bldg]:
        surf_name = res_json[bldg][surf]['name'].upper().replace('-', ' ')
        for index, row in df_eplus_result.iterrows():
            eplus_header_temp = surf_name + ':Surface Outside Face Temperature [C](Hourly)'
            eplus_header_h = surf_name + ':Surface Outside Face Convection Heat Transfer Coefficient [W/m2-K](Hourly)'

            eplus_header_emission_exf = 'Environment:Site Total Zone Exfiltration Heat Loss [J](Hourly)'
            surf_to_zone_rate = res_json[bldg][surf]['area_to_zone']
            
            eplus_header_conv_emission = surf_name + ':Surface Outside Face Heat Emission to Air Rate [W](Hourly)'
            exf_emission = row[eplus_header_emission_exf] * 0.000277778 * surf_to_zone_rate
            
            rel_emission = 0
            rej_emission = 0
            if 'ROOF' in surf_name:
                eplus_header_emission_rej = 'SimHVAC:HVAC System Total Heat Rejection Energy [J](Hourly)'
                eplus_header_emission_rel = 'SimHVAC:Air System Relief Air Total Heat Loss Energy [J](Hourly)'
                rel_emission = row[eplus_header_emission_rel] * 0.000277778
                rej_emission = row[eplus_header_emission_rej] * 0.000277778

                                
            surf_emission_hvac = exf_emission + rel_emission + rej_emission
            surf_emission_total = row[eplus_header_conv_emission] + surf_emission_hvac

            res_json[bldg][surf][row['Date/Time']] = [row[eplus_header_temp], 
                                                      row[eplus_header_h],
                                                      surf_emission_hvac,
                                                      surf_emission_hvac / res_json[bldg][surf]['area'],
                                                      surf_emission_total,
                                                      surf_emission_total / res_json[bldg][surf]['area']
                                                     ]
            time_stamps.add(row['Date/Time'])

In [8]:
### Merging "old" surface to "new" ("new" surfaces represent heat from old surfaces)

In [9]:
base_to_old = {}
with open(os.path.join(scene_folder, 'meta', 'Merged_surfaces.txt'), 'r') as txt_file:
    lines = txt_file.readlines()
for i in range(1, len(lines)):
    old, new = lines[i].strip().split('		')[0].strip(), lines[i].strip().split('		')[1].strip()
    if new not in base_to_old:
        base_to_old[new] = [new, old]
    else:
        base_to_old[new].append(old)

for new_surf in base_to_old['bldg_3981169_2289 :  S1-office-core-wall-0']:
    print(new_surf)

bldg_3981169_2289 :  S1-office-core-wall-0
bldg_3981169_2289 : S1-office-core-wall-1
bldg_3981169_2289 : S1-office-core-wall-10
bldg_3981169_2289 : S1-office-core-wall-11
bldg_3981169_2289 : S1-office-core-wall-2
bldg_3981169_2289 : S1-office-core-wall-3
bldg_3981169_2289 : S2-office-core-wall-1
bldg_3981169_2289 : S2-office-core-wall-10
bldg_3981169_2289 : S2-office-core-wall-11
bldg_3981169_2289 : S2-office-core-wall-2
bldg_3981169_2289 : S2-office-core-wall-3
bldg_3981169_2289 : S3-office-core-wall-10
bldg_3981169_2289 : S3-office-core-wall-11
bldg_3981169_2289 : S3-office-core-wall-2
bldg_3981169_2289 : S4-office-core-wall-10
bldg_3981169_2289 : S4-office-core-wall-11


In [9]:
name_to_id['bldg_3981169_2289']['S1-office-core-wall-0']

'surf_1'

In [10]:
## Before merging the heat and temperatures
# [temperature, conv_coeff, surf_emission_hvac, surf_emission_hvac_flux, surf_emission_total, surf_emission_total_flux]
print(res_json['bldg_3981169_2289']['surf_1'][' 09/01  01:00:00'])

[25.05723901009709, 1.778531810456938, 53.33446209028229, 0.5122835744194743, 61.93245767894831, 0.5948682999492595]


In [11]:
for time in time_stamps:
    for base in base_to_old:
        total_temp, total_conv_coeff, total_q_hvac, total_heat_hvac, total_q, total_heat = 0, 0, 0, 0, 0, 0
        base_bldg = base.split(' : ')[0].strip()
        if base.split(' : ')[1].strip() not in name_to_id[base_bldg]:
            print(base_bldg, base.split(' : ')[1].strip())
            continue
        base_surf = name_to_id[base_bldg][base.split(' : ')[1].strip()]
        total_area = 0
        for old in base_to_old[base]:
            bldg = old.split(' : ')[0].strip()
            surf = name_to_id[bldg][old.split(' : ')[1].strip()]
            curr_area = res_json[bldg][surf]['area']
            total_area += curr_area
            total_temp += res_json[bldg][surf][time][0] * curr_area
            total_conv_coeff += res_json[bldg][surf][time][1] * curr_area
            total_q_hvac += res_json[bldg][surf][time][2] * curr_area
            total_heat_hvac += res_json[bldg][surf][time][3]
            total_q += res_json[bldg][surf][time][4] * curr_area
            total_heat += res_json[bldg][surf][time][5]
        res_json[base_bldg][base_surf][time][0] = total_temp / total_area
        res_json[base_bldg][base_surf][time][1] = total_conv_coeff / total_area
        res_json[base_bldg][base_surf][time][2] = total_q_hvac / total_area
        res_json[base_bldg][base_surf][time][3] = total_heat_hvac
        res_json[base_bldg][base_surf][time][4] = total_q / total_area
        res_json[base_bldg][base_surf][time][5] = total_heat
    
        

In [12]:
## After merging the heat and temperatures
print(res_json['bldg_3981169_2289']['surf_1'][' 09/01  01:00:00'])

[25.657370101904363, 1.6817763869511473, 105.29316109133748, 8.196537190711584, 484.4221199869833, 51.81562176907448]


In [13]:
for time in time_stamps:
    result_json = {}
    result_json['features'] = []
    time_stamp_to_write = time.replace('/', '_').replace('  ', '_').replace(':', '_')[:-3]
    for bldg in res_json:
        result_json_bldg = {}
        result_json_bldg['id'] = bldg
        result_json_bldg['properties'] = {}
        result_json_bldg['properties']['objects'] = []
        for surf in res_json[bldg]:
            result_json_surf = {}
            result_json_surf['id'] = surf
            result_json_surf['properties'] = {}
            result_json_surf['properties']['name'] = res_json[bldg][surf]['name']
            result_json_surf['properties']['temperature'] = res_json[bldg][surf][time][0]
            result_json_surf['properties']['conv_coeff'] = res_json[bldg][surf][time][1]
            result_json_surf['properties']['q_hvac'] = res_json[bldg][surf][time][2]
            result_json_surf['properties']['heat_flux_hvac'] = res_json[bldg][surf][time][3]
            result_json_surf['properties']['q_total'] = res_json[bldg][surf][time][4]
            result_json_surf['properties']['heat_flux_total'] = res_json[bldg][surf][time][5]
            result_json_bldg['properties']['objects'].append(result_json_surf)
        result_json['features'].append(result_json_bldg)
    ## TODO
    file = open(os.path.join(scene_folder, '2-eplus', 'sf_100_dt_hvac_eplus_2_' + time_stamp_to_write.strip() + '.json'), 'w')
    file.write(json.dumps(result_json, indent=2))
    file.close()

In [14]:
result_json

{'features': [{'id': 'bldg_3981169_2289',
   'properties': {'objects': [{'id': 'surf_1',
      'properties': {'name': 'S1-office-core-wall-0',
       'temperature': 26.286032462896248,
       'conv_coeff': 2.662666964658598,
       'q_hvac': -3.8204831765511393,
       'heat_flux_hvac': -0.2974051886990563,
       'q_total': -4328.38012896251,
       'heat_flux_total': -180.3296046378921}},
     {'id': 'surf_2',
      'properties': {'name': 'S1-office-core-wall-1',
       'temperature': 24.857095402779773,
       'conv_coeff': 2.1550884556860708,
       'q_hvac': -0.5181573221692318,
       'heat_flux_hvac': -0.01858782429369102,
       'q_total': -189.88077682704022,
       'heat_flux_total': -6.811580895228273}},
     {'id': 'surf_3',
      'properties': {'name': 'S1-office-core-wall-10',
       'temperature': 25.07792728822893,
       'conv_coeff': 2.493938089079351,
       'q_hvac': -1.9580621034433956,
       'heat_flux_hvac': -0.01858782429369102,
       'q_total': -1315.72806988